# 搜索引擎评价
## 效果评价
### 召回率和准确率
- A是相关文档集合，B是被检索到的文档集合
- 召回率$R=\frac{|A \cap B|}{|A|}$
- 准确率$P=\frac{|A \cap B|}{|B|}$
- 虚报率$=\frac{|\bar{A} \cap B|}{|\bar{A}|}$
- $F=\frac{1}{\frac{1}{2}(\frac{1}{R}+\frac{1}{P})}=\frac{2RP}{R+P}$
- 位置p的准确率（precision of p）：如果一个排序位置p的准确率比另一个排序高，召回率也会较高。
  - 最常用的P@10和P@20的准确率度量。
  - 暗示搜索任务已经变为在给定排序中找到尽可能相关的文档，而不是找到尽可能多的相关文档。
- 当召回率从0.0到1.0每次增加0.1这一标准值时，计算准确率的相应变化。
  - 优点在于，可以用于评价所有排序结果中的相关文档，而不仅仅是那些靠前的文档。
- 计算当一个额外的相关文档被检出（也就是当召回率增加）时，准确率的平均值。

### 平均化和插值
- 平均准确率(Mean Average Precision,MAP):平均准确率为每个查询的相关排序结果赋予一个评价数字，从多个查询中总结该排序算法性能的最简单的途径，就是对这些数字进行平均。
- 标准召回率等级是0.0到1.0，增量为0.1.为了获得每个查询在这些召回率等级下的准确率，必须进行插值。

### 关注排序靠前的文档
- 排序倒数(reciprocal rank)定义为返回第一个相关文档位置的倒数。平均排序倒数MRR(mean reciprocal rank)是针对一组查询的排序倒数平均值。
- 对于网络搜索而言，DCG(discounted cumulative gain)是一种较为普遍的评价方法。这种方法基于两点假设：
  - 高相关的文档比边缘相关的文档要有用的多。
  - 一个相关文档的排序位置越靠后，对于用户的价值就越低，因为它们很少会被用户查到。
  - DCG方法是在一个特定的排序p的前提下，计算总的增益：$DCG_p=rel_1+\sum_{i=2}^p\frac{rel_i}{log_2i}$
    - $rel_i$指的是检索回的文档中排序为i的文档的相关性等级。
    - $log_2i$是一种损失因子(discount or reduction factor)。

### 使用用户偏好
- Kendall $\tau$系数
  - 使用偏好描述的两个排序结果，P是两种排序中一致的偏好的数量，Q代表不一致的数量。
  - 这个估计值在1（当两个排序中用户偏好全部一致时）到-1（当偏好全部不一致时）之间变化。
  - $\tau=\frac{P-Q}{P+Q}$
- BPREF
  - 偏好来自二元相关判决的情况。
  - 对于一个返回了R个相关文档的查询，仅考察前R个不相关文档，相当于使用RxR个用户偏好。
  - $BPREF=\frac{1}{R}\sum_{d_r}(1-\frac{N_{d_r}}{R})$
    - $d_r$是一个相关文档，$N_{d_r}$给出了排序高于$d_r$的不相关文档的数量（考察不相关文档集R）
    - 另一种定义$BPREF=\frac{P}{P+Q}$

## 效率评价
- 基于`查询流量(query throughput)`的方法，记为每秒处理的查询数量，它并不捕捉`延迟(latency)`。

## 训练、测试和统计
### 显著性检验
使用某种评价方法对比两个不同检索算法产生的排序结果。零假设：两个检索算法在效果评价方面有什么差别。

表8-6 针对10个查询，“B-A”列出两个算法之间效果评价值的不同

| 查询 | A | B | B-A |
| --- | -- | -- | -- |
| 1 | 25 | 35 | 10 |
| 2 | 43 | 84 | 41 |
| 3 | 39 | 15 | -24 |
| 4 | 75 | 75 | 0 |
| 5 | 43 | 68 | 25 |
| 6 | 15 | 85 | 70 |
| 7 | 20 | 80 | 60 |
| 8 | 52 | 50 | -2 |
| 9 | 49 | 58 | 9 |
| 10 | 50 | 75 | 25 |

#### t检验(t-test)
t检验首先假定采样的数值符合正态分布。这种假设指的是这两种算法的效果评价值之间的差值是正态分布的一个样例。这种情况下的零假设是指效果评价值之间的差值分布的平均值为0。

- $\bar{B-A}=21.4$
- $\sigma_{B-A}=29.1$
- $t=2.33$

对于单侧检验，p-value=0.02，这在假设$\alpha=0.05$时，可以证明这两个算法的差别是显著的。因此，t检验可以否定零假设，并得出B比A有效的论证。

#### 威氏符号秩次检验(Wilcoxon signed-rank test)
假定算法A和B之间的效果评价值的差值可以被排序，但是差值的尺度并不重要。

表8-6 9个非0的差值，将它们的绝对值进行排序：

```
2,9,10,24,25,25,41,60,70
```

相应的符号秩次为：

```
-1,+2,+3,-4,+5.5,+5.5,+7,+8,+9
```

排序值累加，得到w=35。对于单侧检验而言，获得p-value=0.025，这在假设$\alpha=0.05$时，否定零假设。

#### 符号检验(sign test)
零假设是指$P(B>A)=P(A>B)=\frac{1}{2}$，这意味着显示B比A“好”的数据对的数量和A比B“好”的数据对的数量是相同的。


### 设置参数值
- 交叉检验(cross-validation)
- 强制法(brute-force)，穷举可能的参数
- 最优化技术

### 在线测试
- 优点
  - 允许真实用户参与到系统中。
  - 并不片面，这是因为这种评价方法是在真实的用户数据集上进行的。
- 缺陷
  - 采集的数据噪声比较大。
  - 使得实时通讯量以一种不利的方式进行潜在的改变（有可能大幅度降低检索的效果，从而导致用户的离开）。
